# Football Analysis (2012 to 2023) Data engineering project 

#### GET Postgres to work on Python

In [1]:
import psycopg2 #Postgres with Python


#### Establishing the connection with Postgres and Creating a new Database

In [3]:
try: 
    conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=postgres password=Hammaad@Rizwan")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)#Establishing conneion with the database
    
try: 
    cur = conn.cursor()#Cursor is a method tat allows us to use the database.Allows Python code to execute PostgreSQL command in a database session
except psycopg2.Error as e: 
    print("Error: Could not get curser to the Database")
    print(e)

conn.set_session(autocommit=True)#like autosave
try: 
    cur.execute("create database football_Analysis")#Creates a table called football analysis
except psycopg2.Error as e:
    print(e)

database "football_analysis" already exists



#### Closes the main connection, and reopens it in the new Database

In [4]:
try: 
    conn.close()#Closes the connection
except psycopg2.Error as e:
    print(e)
    
try: #Reopens the connection under the new database
    conn = psycopg2.connect("host=127.0.0.1 dbname=football_analysis user=postgres password=Hammaad@Rizwan")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)
    
try: 
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get curser to the Database")
    print(e)

#### Creating 4 tables


4 Tables have been created with the relavant Primary and forieng keys 

In [25]:
try: 
    conn = psycopg2.connect("host=127.0.0.1 dbname=football_analysis user=postgres password=Hammaad@Rizwan")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)#Establishing conneion with the database
    
try: 
    cur = conn.cursor()#Cursor is a method tat allows us to use the database.Allows Python code to execute PostgreSQL command in a database session
except psycopg2.Error as e: 
    print("Error: Could not get curser to the Database")
    print(e)

conn.set_session(autocommit=True)#like autosave
try: 
    cur.execute("CREATE TABLE competitions(competition_ID VARCHAR, Name VARCHAR, type VARCHAR,sub_type VARCHAR,PRIMARY KEY(competition_ID));")
    cur.execute("CREATE TABLE clubs(club_ID int,Name VARCHAR,competition_ID VARCHAR,squad_size int,PRIMARY KEY(club_ID));")
    cur.execute("CREATE TABLE players(player_ID int,club_ID int,Date_of_birth VARCHAR,sub_position VARCHAR, foot VARCHAR, height int,competition_ID VARCHAR,First_name VARCHAR,Last_name VARCHAR,PRIMARY KEY(player_ID));")
    cur.execute("CREATE TABLE GAMES(game_ID int,competition_ID VARCHAR, Date VARCHAR, Home_club_ID int,Away_club_ID int,Home_club_goals int,Away_club_goals int,PRIMARY KEY(game_ID));")

except psycopg2.Error as e:
    print(e)

## Reading files using Panda

In [6]:
import pandas as pd

In [22]:
competitions = pd.read_csv("competitons.csv")
clubs = pd.read_csv("C:\\Users\\Hammaad\\Documents\\clubs.csv")
players = pd.read_csv("C:\\Users\\Hammaad\\Documents\\players.csv")
games = pd.read_csv("game.csv")

## Cleaning data

In [9]:
#competition data is clean, no missing values

,player_id,current_club_id,date_of_birth,sub_position,foot,height_in_cm,current_club_domestic_competition_id,first_name,last_name
28366,452068,15,10 04 2001,Defensive Midfield,NaN,175,L1,Ayman,Azhil
28367,85543,15,28 07 1993,NaN,Right,186,L1,Niklas,Lomb
28368,315131,15,2 01 1998,Right-Back,Right,185,L1,Timothy,Fosu-Mensah
28369,39728,16,16 12 1988,Centre-Back,Right,191,L1,Mats,Hummels
28370,659813,15,9 01 2002,Centre-Back,Left,183,L1,Piero,Hincapié


In [106]:

pos=players["first_name"].mode()[0]
for i in range(len(players["first_name"])):
    if players["first_name"][i]=="nan":
        players["first_name"][i]=players["first_name"].mode()[0]

    

## Reading the csv file using pandas and then by iterating each row we insert it to the relavant table

#### Populating competitions table

In [14]:
try:
    # Establish a connection to your PostgreSQL database
    conn = psycopg2.connect("host=127.0.0.1 dbname=football_analysis user=postgres password=Hammaad@Rizwan")
    cur = conn.cursor()

    # Begin a new transaction
    cur.execute("BEGIN")

    competitions_insert = ("""INSERT INTO competitions(
    competition_id ,
    name ,
    type ,
    sub_type)  VALUES(%s,%s,%s,%s)""")
    #games_insert = ("""INSERT INTO games(Game_ID,competition_ID , Date , Home_club_ID ,Away_club_ID ,Home_club_goals ,Away_club_goals ) VALUES(%s,%s,%s,%s,%s,%s,%s)""")
    
    
    # Iterate over the rows of the 'competitions' DataFrame and execute the SQL statement
 
    for i, row in competitions.iterrows():
        cur.execute(competitions_insert, list(row))
    cur.execute("COMMIT")

except psycopg2.DatabaseError as e:
    # Rollback the transaction and handle the error
    conn.rollback()
    print("Transaction failed and rolled back:", str(e))

finally:
    # Close the cursor and connection
    if cur:
        cur.close()
    if conn:
        conn.close()

#### Populating clubs table

In [43]:
try:
    # Establish a connection to your PostgreSQL database
    conn = psycopg2.connect("host=127.0.0.1 dbname=football_analysis user=postgres password=Hammaad@Rizwan")
    cur = conn.cursor()

    # Begin a new transaction
    cur.execute("BEGIN")

    clubs_insert = ("""INSERT INTO clubs(
    club_id ,
    name ,
    competition_id ,
    squad_size)  VALUES(%s,%s,%s,%s)""")

    # Iterate over the rows of the 'clubs' DataFrame and execute the SQL statement
 
    for i, row in clubs.iterrows():
        cur.execute(clubs_insert, list(row))
    cur.execute("COMMIT")

except psycopg2.DatabaseError as e:
    # Rollback the transaction and handle the error
    conn.rollback()
    print("Transaction failed and rolled back:", str(e))

finally:
    # Close the cursor and connection
    if cur:
        cur.close()
    if conn:
        conn.close()

#### Populating players table

In [107]:
try:
    # Establish a connection to your PostgreSQL database
    conn = psycopg2.connect("host=127.0.0.1 dbname=football_analysis user=postgres password=Hammaad@Rizwan")
    cur = conn.cursor()

    # Begin a new transaction
    cur.execute("BEGIN")

    players_insert = ("""INSERT INTO players(
    player_id ,
    club_id ,
    date_of_birth ,
    sub_position,
    foot,
    height,
    competition_id,
    first_name,
    last_name)  VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s)""")

    # Iterate over the rows of the 'clubs' DataFrame and execute the SQL statement
 
    for i, row in players.iterrows():
        cur.execute(players_insert, list(row))
    cur.execute("COMMIT")

except psycopg2.DatabaseError as e:
    # Rollback the transaction and handle the error
    conn.rollback()
    print("Transaction failed and rolled back:", str(e))

finally:
    # Close the cursor and connection
    if cur:
        cur.close()
    if conn:
        conn.close()

#### Populating games table

In [118]:
try:
    # Establish a connection to your PostgreSQL database
    conn = psycopg2.connect("host=127.0.0.1 dbname=football_analysis user=postgres password=Hammaad@Rizwan")
    cur = conn.cursor()

    # Begin a new transaction
    cur.execute("BEGIN")

    games_insert = ("""INSERT INTO games(
    game_id ,
    competition_id ,
    date ,
    home_club_id,
    away_club_id,
    home_club_goals,
    away_club_goals)  VALUES(%s,%s,%s,%s,%s,%s,%s)""")

    # Iterate over the rows of the 'clubs' DataFrame and execute the SQL statement
 
    for i, row in games.iterrows():
        cur.execute(games_insert, list(row))
    cur.execute("COMMIT")

except psycopg2.DatabaseError as e:
    # Rollback the transaction and handle the error
    conn.rollback()
    print("Transaction failed and rolled back:", str(e))

finally:
    # Close the cursor and connection
    if cur:
        cur.close()
    if conn:
        conn.close()